# Join Data Banjir dengan Data Kecamatan

Notebook ini akan melakukan join antara:
- `data_banjir_combine_final.csv` (18,048 baris) 
- `kecamatan_filtered.csv` (7,287 baris)

Berdasarkan nama kecamatan dengan normalisasi text untuk menangani perbedaan format.

In [12]:
import pandas as pd
import numpy as np
import re
from unicodedata import normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Data

In [13]:
df_banjir = pd.read_csv('data_banjir_combine_final.csv')
df_kecamatan = pd.read_csv('kecamatan_filtered.csv')

print(df_banjir.head(3))

print(df_kecamatan.head(3))

print(df_kecamatan.columns.tolist())

  NAME_2         NAME_3  avg_rainfall  max_rainfall  avg_temperature  \
0  bogor  babakanmadang        64.775       124.032            28.76   
1  bogor     bojonggede        68.241       128.949            32.89   
2  bogor       caringin        56.423        92.324            29.93   

   elevation landcover_class   ndvi  slope  soil_moisture  year  month  \
0    501.913      Tree cover  0.564  5.001         39.583  2020      1   
1    133.589        Built-up  0.429  0.293         40.414  2020      1   
2    765.651      Tree cover  0.513  4.954         39.662  2020      1   

   banjir     lat      long                  map_image   NAME_3_clean  
0       1 -6.5917  106.9020  RGB_BabakanMadang2020.tif  babakanmadang  
1       1 -6.4774  106.7927  RGB_BabakanMadang2020.tif     bojonggede  
2       0 -6.7366  106.8660    RGB_Caringin2020(2).tif       caringin  
   objectid  kode_kec_spatial            nama_prop         nama_kab  \
0       NaN          130515.0       SUMATERA BARAT  PAD

## 2. Fungsi Normalisasi Nama Kecamatan

Membuat fungsi untuk menormalisasi nama kecamatan agar matching lebih akurat:
- Convert ke lowercase
- Hapus spasi berlebih
- Hapus karakter khusus
- Normalisasi unicode (untuk karakter Indonesia)

In [ ]:
def normalize_kecamatan_name(name):
    """
    Normalisasi nama kecamatan untuk matching yang lebih baik
    
    Args:
        name: nama kecamatan (string atau NaN)
    
    Returns:
        normalized string atau empty string jika NaN
    """
    if pd.isna(name):
        return ''
    
    name = str(name)
    name = normalize('NFD', name).encode('ascii', 'ignore').decode('utf-8')
    name = name.lower()
    name = re.sub(r'[^a-z0-9\s]', '', name)
    name = re.sub(r'\s+', '', name)
    
    return name.strip()

test_names = [
    'Babakan Madang',
    'BOJONGGEDE',
    'Cibinong',
    'Kota Tangerang',
    '2 X 11 KAYU TANAM',
    None,
    ''
]

for test in test_names:
    print(f"{str(test):30s} -> {normalize_kecamatan_name(test)}")

Babakan Madang                 -> babakanmadang
BOJONGGEDE                     -> bojonggede
Cibinong                       -> cibinong
Kota Tangerang                 -> kotatangerang
2 X 11 KAYU TANAM              -> 2x11kayutanam
None                           -> 
                               -> 


## 3. Apply Normalisasi ke Kedua Dataset

In [ ]:
df_banjir['kecamatan_key'] = df_banjir['NAME_3_clean'].apply(normalize_kecamatan_name)
df_kecamatan['kecamatan_key'] = df_kecamatan['nama_kec'].apply(normalize_kecamatan_name)

print(df_banjir[['NAME_3', 'NAME_3_clean', 'kecamatan_key']].head(10))
print(df_kecamatan[['nama_kec', 'kecamatan_key']].head(10))

print(f"Empty keys in df_banjir: {(df_banjir['kecamatan_key'] == '').sum()}")
print(f"Empty keys in df_kecamatan: {(df_kecamatan['kecamatan_key'] == '').sum()}")

          NAME_3   NAME_3_clean  kecamatan_key
0  babakanmadang  babakanmadang  babakanmadang
1     bojonggede     bojonggede     bojonggede
2       caringin       caringin       caringin
3          cariu          cariu          cariu
4        ciampea        ciampea        ciampea
5          ciawi          ciawi          ciawi
6       cibinong       cibinong       cibinong
7   cibungbulang   cibungbulang   cibungbulang
8      cigombong      cigombong      cigombong
9        cigudeg        cigudeg        cigudeg
            nama_kec  kecamatan_key
0  2 X 11 KAYU TANAM  2x11kayutanam
1       ABAD SELATAN    abadselatan
2              ABANG          abang
3            ABEPURA        abepura
4         ABIANSEMAL     abiansemal
5               ABOY           aboy
6              ABUKI          abuki
7               ABUN           abun
8        ABUNG BARAT     abungbarat
9       ABUNG KUNANG    abungkunang
Empty keys in df_banjir: 7067
Empty keys in df_kecamatan: 39


## 4. Analisis Overlap Sebelum Join

Cek berapa banyak kecamatan yang ada di kedua dataset

In [16]:
unique_banjir = set(df_banjir['kecamatan_key'].unique())
unique_kecamatan = set(df_kecamatan['kecamatan_key'].unique())

unique_banjir.discard('')
unique_kecamatan.discard('')

overlap = unique_banjir.intersection(unique_kecamatan)
only_in_banjir = unique_banjir - unique_kecamatan
only_in_kecamatan = unique_kecamatan - unique_banjir

print(f"Unique kecamatan di df_banjir: {len(unique_banjir)}")
print(f"Unique kecamatan di df_kecamatan: {len(unique_kecamatan)}")
print(f"Kecamatan yang ada di KEDUA dataset: {len(overlap)}")
print(f"Kecamatan HANYA di df_banjir: {len(only_in_banjir)}")
print(f"Kecamatan HANYA di df_kecamatan: {len(only_in_kecamatan)}")


for i, kec in enumerate(sorted(list(overlap))[:20], 1):
    print(f"{i:3d}. {kec}")

if len(only_in_banjir) > 0:
    for i, kec in enumerate(sorted(list(only_in_banjir))[:10], 1):
        original = df_banjir[df_banjir['kecamatan_key'] == kec]['NAME_3_clean'].iloc[0]
        print(f"{i:3d}. {original:30s} -> {kec}")

Unique kecamatan di df_banjir: 180
Unique kecamatan di df_kecamatan: 6846
Kecamatan yang ada di KEDUA dataset: 180
Kecamatan HANYA di df_banjir: 0
Kecamatan HANYA di df_kecamatan: 6666
  1. babakanmadang
  2. babelan
  3. balaraja
  4. bantargebang
  5. batuceper
  6. beji
  7. bekasibarat
  8. bekasiselatan
  9. bekasitimur
 10. bekasiutara
 11. benda
 12. bogorbarat
 13. bogorselatan
 14. bogortengah
 15. bogortimur
 16. bogorutara
 17. bojonggede
 18. bojongmangu
 19. bojongsari
 20. cabangbungin


## 5. Perform Join

Gunakan **left join** untuk mempertahankan semua data dari df_banjir dan tambahkan informasi dari df_kecamatan jika ada.

In [ ]:
print(f"Duplikasi di df_banjir: {df_banjir['kecamatan_key'].duplicated().sum()}")
print(f"Duplikasi di df_kecamatan: {df_kecamatan['kecamatan_key'].duplicated().sum()}")

if df_kecamatan['kecamatan_key'].duplicated().sum() > 0:
    print("\nDitemukan duplikasi di df_kecamatan!")
    print("Duplikat kecamatan:")
    dupes = df_kecamatan[df_kecamatan['kecamatan_key'].duplicated(keep=False)].sort_values('kecamatan_key')
    print(dupes[['nama_prop', 'nama_kab', 'nama_kec', 'kecamatan_key']].head(20))
    
    print("\nMenghapus duplikasi di df_kecamatan (keep='first')...")
    df_kecamatan_clean = df_kecamatan.drop_duplicates(subset='kecamatan_key', keep='first')
    print(f"Setelah deduplikasi: {len(df_kecamatan_clean)} rows")
else:
    df_kecamatan_clean = df_kecamatan.copy()
    print("\nTidak ada duplikasi di df_kecamatan")

Duplikasi di df_banjir: 17866
Duplikasi di df_kecamatan: 438

Ditemukan duplikasi di df_kecamatan!
Duplikat kecamatan:
     nama_prop nama_kab nama_kec kecamatan_key
18         NaN      NaN      NaN              
698        NaN      NaN      NaN              
1084       NaN      NaN      NaN              
1694       NaN      NaN      NaN              
1876       NaN      NaN      NaN              
2032       NaN      NaN      NaN              
5261       NaN      NaN      NaN              
2371       NaN      NaN      NaN              
2403       NaN      NaN      NaN              
2404       NaN      NaN      NaN              
2405       NaN      NaN      NaN              
622        NaN      NaN      NaN              
2424       NaN      NaN      NaN              
3003       NaN      NaN      NaN              
3035       NaN      NaN      NaN              
4256       NaN      NaN      NaN              
4267       NaN      NaN      NaN              
4288       NaN      NaN      NaN   

In [ ]:
df_joined = df_banjir.merge(
    df_kecamatan_clean,
    on='kecamatan_key',
    how='left',
    suffixes=('_banjir', '_kecamatan')
)

print(f"Shape df_banjir (original): {df_banjir.shape}")
print(f"Shape df_kecamatan (cleaned): {df_kecamatan_clean.shape}")
print(f"Shape df_joined (result): {df_joined.shape}")

print(df_joined.head())

Shape df_banjir (original): (18047, 18)
Shape df_kecamatan (cleaned): (6847, 22)
Shape df_joined (result): (18047, 39)
  NAME_2         NAME_3  avg_rainfall  max_rainfall  avg_temperature  \
0  bogor  babakanmadang        64.775       124.032            28.76   
1  bogor     bojonggede        68.241       128.949            32.89   
2  bogor       caringin        56.423        92.324            29.93   
3  bogor          cariu        60.034        98.129            30.02   
4  bogor        ciampea        66.380       133.877            30.27   

   elevation landcover_class   ndvi  slope  soil_moisture  year  month  \
0    501.913      Tree cover  0.564  5.001         39.583  2020      1   
1    133.589        Built-up  0.429  0.293         40.414  2020      1   
2    765.651      Tree cover  0.513  4.954         39.662  2020      1   
3    143.845      Tree cover  0.652  2.107         39.014  2020      1   
4    212.619      Tree cover  0.569  1.340         39.658  2020      1   

   

## 6. Validasi Hasil Join

In [ ]:
if 'nama_kab' in df_joined.columns:
    matched_rows = df_joined['nama_kab'].notna().sum()
    unmatched_rows = df_joined['nama_kab'].isna().sum()
    
    print(f"Total baris: {len(df_joined)}")
    print(f"Baris yang berhasil di-join: {matched_rows} ({matched_rows/len(df_joined)*100:.2f}%)")
    print(f"Baris yang TIDAK ter-join: {unmatched_rows} ({unmatched_rows/len(df_joined)*100:.2f}%)")
    
    if unmatched_rows > 0:
        unmatched = df_joined[df_joined['nama_kab'].isna()]
        print(unmatched[['NAME_2', 'NAME_3', 'NAME_3_clean', 'kecamatan_key']].drop_duplicates().head(10))
else:
    print("Kolom 'nama_kab' tidak ditemukan. Cek nama kolom!")

kecamatan_cols = [col for col in df_joined.columns if col in df_kecamatan.columns or '_kecamatan' in col]
for col in kecamatan_cols[:10]:
    missing = df_joined[col].isna().sum()
    print(f"{col:40s}: {missing:6d} ({missing/len(df_joined)*100:5.2f}%)")

Total baris: 18047
Baris yang berhasil di-join: 10980 (60.84%)
Baris yang TIDAK ter-join: 7067 (39.16%)
             NAME_2        NAME_3 NAME_3_clean kecamatan_key
10980  Aceh Selatan   kluettengah          NaN              
10981  Aceh Selatan    kluettimur          NaN              
10982  Aceh Selatan        meukek          NaN              
10983  Aceh Selatan       samadua          NaN              
10984  Aceh Selatan     tapaktuan          NaN              
10985  Aceh Selatan        trumon          NaN              
10986  Aceh Singkil  simpangkanan          NaN              
10987          Agam   kamangmagek          NaN              
10988          Agam   lubukbasung          NaN              
10989          Agam    palembayan          NaN              
kecamatan_key                           :      0 ( 0.00%)
objectid                                :  18047 (100.00%)
kode_kec_spatial                        :   7067 (39.16%)
nama_prop                               :   7067 (

## 7. Clean Up & Export

Hapus kolom helper (kecamatan_key) dan export hasil join

In [ ]:
df_final = df_joined.copy()
if 'kecamatan_key' in df_final.columns:
    df_final = df_final.drop(columns=['kecamatan_key'])
    print("Kolom 'kecamatan_key' telah dihapus")

print(f"Shape: {df_final.shape}")
print(f"Jumlah kolom: {len(df_final.columns)}")
print(f"\nKolom-kolom:")
for i, col in enumerate(df_final.columns, 1):
    print(f"{i:3d}. {col}")

print(df_final.describe())

Kolom 'kecamatan_key' telah dihapus
Shape: (18047, 38)
Jumlah kolom: 38

Kolom-kolom:
  1. NAME_2
  2. NAME_3
  3. avg_rainfall
  4. max_rainfall
  5. avg_temperature
  6. elevation
  7. landcover_class
  8. ndvi
  9. slope
 10. soil_moisture
 11. year
 12. month
 13. banjir
 14. lat
 15. long
 16. map_image
 17. NAME_3_clean
 18. objectid
 19. kode_kec_spatial
 20. nama_prop
 21. nama_kab
 22. nama_kec
 23. jumlah_penduduk
 24. kepadatan_penduduk
 25. jumlah_kk
 26. luas_wilayah
 27. pertumbuhan_2022
 28. pertumbuhan_2021
 29. pertumbuhan_2020
 30. perpindahan_pddk
 31. jumlah_desa
 32. jumlah_kelurahan
 33. perdagangan
 34. nelayan
 35. wiraswasta
 36. tidak_blm_sekolah
 37. slta
 38. s1
       avg_rainfall  max_rainfall  avg_temperature     elevation  \
count  18047.000000  18047.000000     18047.000000  18047.000000   
mean      44.969022     83.676593        32.182496    155.967521   
std       25.480935     44.370820         4.676819    256.414214   
min        0.525000      0.85

In [ ]:
output_filename = 'data_banjir_joined.csv'

try:
    df_final.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"Data berhasil disimpan ke: {output_filename}")
    print(f"  - Total rows: {len(df_final):,}")
    print(f"  - Total columns: {len(df_final.columns)}")
    import os
    file_size = os.path.getsize(output_filename) / (1024 * 1024)
    print(f"  - File size: {file_size:.2f} MB")
    
except Exception as e:
    print(f"Error saat menyimpan file: {e}")

✓ Data berhasil disimpan ke: data_banjir_joined.csv
  - Total rows: 18,047
  - Total columns: 38
  - File size: 3.88 MB


In [ ]:
df_joined_clean = pd.read_csv('data_banjir_joined.csv')

print(f"Shape: {df_joined_clean.shape}")
print(f"\nKolom yang akan diubah:")
print(f"  - NAME_2: {df_joined_clean['NAME_2'].iloc[0]}")
print(f"  - NAME_3: {df_joined_clean['NAME_3'].iloc[0]}")
print(f"  - NAME_3_clean: {df_joined_clean['NAME_3_clean'].iloc[0]}")

if 'objectid' in df_joined_clean.columns:
    null_count = df_joined_clean['objectid'].isna().sum()
    total_count = len(df_joined_clean)
    print(f"\n  - objectid: {null_count}/{total_count} kosong ({null_count/total_count*100:.1f}%)")
else:
    print("\n  - objectid: kolom tidak ditemukan")

Shape: (18047, 38)

Kolom yang akan diubah:
  - NAME_2: bogor
  - NAME_3: babakanmadang
  - NAME_3_clean: babakanmadang

  - objectid: 18047/18047 kosong (100.0%)


C:\Users\admin\AppData\Local\Temp\ipykernel_19152\1395644699.py:1: DtypeWarning: Columns (16,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_joined_clean = pd.read_csv('data_banjir_joined.csv')


In [ ]:
rename_dict = {
    'NAME_2': 'kabupaten_kota',      
    'NAME_3': 'kecamatan',            
    'NAME_3_clean': 'kecamatan_clean' 
}

df_joined_clean = df_joined_clean.rename(columns=rename_dict)

for old_name, new_name in rename_dict.items():
    print(f"  {old_name:20s} -> {new_name}")

if 'objectid' in df_joined_clean.columns:
    df_joined_clean = df_joined_clean.drop(columns=['objectid'])
    print("\n  Kolom 'objectid' telah dihapus")
else:
    print("\n  Kolom 'objectid' tidak ditemukan")


print(f"Shape: {df_joined_clean.shape}")
print(f"\nSample data dengan nama kolom baru:")
print(df_joined_clean[['kabupaten_kota', 'kecamatan', 'kecamatan_clean']].head())

  ✓ NAME_2               -> kabupaten_kota
  ✓ NAME_3               -> kecamatan
  ✓ NAME_3_clean         -> kecamatan_clean

  ✓ Kolom 'objectid' telah dihapus
Shape: (18047, 37)

Sample data dengan nama kolom baru:
  kabupaten_kota      kecamatan kecamatan_clean
0          bogor  babakanmadang   babakanmadang
1          bogor     bojonggede      bojonggede
2          bogor       caringin        caringin
3          bogor          cariu           cariu
4          bogor        ciampea         ciampea


In [ ]:
for i, col in enumerate(df_joined_clean.columns, 1):
    if col in ['kabupaten_kota', 'kecamatan', 'kecamatan_clean']:
        print(f"{i:3d}. {col} *")
    else:
        print(f"{i:3d}. {col}")

print(f"\nTotal kolom: {len(df_joined_clean.columns)}")
print(f"Total baris: {len(df_joined_clean):,}")

  1. kabupaten_kota ⭐
  2. kecamatan ⭐
  3. avg_rainfall
  4. max_rainfall
  5. avg_temperature
  6. elevation
  7. landcover_class
  8. ndvi
  9. slope
 10. soil_moisture
 11. year
 12. month
 13. banjir
 14. lat
 15. long
 16. map_image
 17. kecamatan_clean ⭐
 18. kode_kec_spatial
 19. nama_prop
 20. nama_kab
 21. nama_kec
 22. jumlah_penduduk
 23. kepadatan_penduduk
 24. jumlah_kk
 25. luas_wilayah
 26. pertumbuhan_2022
 27. pertumbuhan_2021
 28. pertumbuhan_2020
 29. perpindahan_pddk
 30. jumlah_desa
 31. jumlah_kelurahan
 32. perdagangan
 33. nelayan
 34. wiraswasta
 35. tidak_blm_sekolah
 36. slta
 37. s1

Total kolom: 37
Total baris: 18,047


## 9. Export Data Final yang Sudah Dibersihkan

In [ ]:
output_filename_final = 'data_banjir_joined_clean.csv'

try:
    df_joined_clean.to_csv(output_filename_final, index=False, encoding='utf-8-sig')
    print(f"File: {output_filename_final}")
    print(f"Total rows: {len(df_joined_clean):,}")
    print(f"Total columns: {len(df_joined_clean.columns)}")
    
    import os
    file_size = os.path.getsize(output_filename_final) / (1024 * 1024)
    print(f"File size: {file_size:.2f} MB")
    
    print("  NAME_2 -> kabupaten_kota")
    print("  NAME_3 -> kecamatan")
    print("  NAME_3_clean -> kecamatan_clean")
    print("  Kolom 'objectid' dihapus")
    
except Exception as e:
    print(f"Error saat menyimpan file: {e}")

File: data_banjir_joined_clean.csv
Total rows: 18,047
Total columns: 37
File size: 3.87 MB
  ✓ NAME_2 → kabupaten_kota
  ✓ NAME_3 → kecamatan
  ✓ NAME_3_clean → kecamatan_clean
  ✓ Kolom 'objectid' dihapus


In [ ]:
print(df_joined_clean.head(10))

print(df_joined_clean.info())

  kabupaten_kota      kecamatan  avg_rainfall  max_rainfall  avg_temperature  \
0          bogor  babakanmadang        64.775       124.032            28.76   
1          bogor     bojonggede        68.241       128.949            32.89   
2          bogor       caringin        56.423        92.324            29.93   
3          bogor          cariu        60.034        98.129            30.02   
4          bogor        ciampea        66.380       133.877            30.27   
5          bogor          ciawi        60.033       102.824            30.31   
6          bogor       cibinong        64.197       120.134            32.59   
7          bogor   cibungbulang        62.757       125.628            30.10   
8          bogor      cigombong        53.486        84.206            27.78   
9          bogor        cigudeg        61.410       109.292            28.26   

   elevation landcover_class   ndvi  slope  soil_moisture  year  month  \
0    501.913      Tree cover  0.564  5.001   